# cnn-emb 데모
- embedding :: 각 단어를 정수를 인코딩하여 정수를 실수 벡터로 변환하는 것  
- 주로 신경망으로 학습하며 층에대한 아웃풋을 그 다음 층의 임베딩(입력)응로 사용하여 학습 

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [2]:
data_dir    = Path('C:\\Users\\USER\\Desktop\\open\\')
tst_dir     = Path('C:\\Users\\USER\\Desktop\\open\\')
feature_dir = Path('C:\\Users\\USER\\Desktop\\open\\feature\\')
sub_dir     = Path('C:\\Users\\USER\\Desktop\\open\\sub\\')
val_dir     = Path('C:\\Users\\USER\\Desktop\\open\\val\\')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

algo_name = 'cnn'
feature_name = 'emb'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

## train 데이터 살펴보기

In [3]:
train = pd.read_csv(trn_file, encoding = 'utf-8')
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


## test 데이터 살펴보기

In [4]:
test = pd.read_csv(tst_file, encoding = 'utf-8')
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


## Preprocessing

In [5]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [6]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [7]:
# train test 분리
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


In [8]:
X_train[:3]

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said'],
      dtype=object)

## Training
- 보통 미리 학습된 임베딩 오픈소스 사용-->장점:단어수 (데이터량)에 영향 받지 않고 다양한 단어, 많은 단어로 학습 진행가능
- 추가로 보통 오픈소스 300개 유의미 한 관계추출 가능 적은 데이터 만으로도 무게는 고정하여 나머지 측의 무게를 변환하여 학습 가능
1. word3vec by google 두개 다른 뉴런 네트워크 구조를 사용 
2. fastText by facebook 단어 개수를 가지고 나눌때 단위를 단순ㅅ 수가 아닌 캐릭터 단위로 나누어 범주를 만듬
3. GloVe by Stanford 자주 출현한 빈도를 계산하는 것을 도와줌(강사님 추천!!)

In [9]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 64 
max_length = 500
padding_type='post'

In [10]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [11]:
#데이터를 sequence로 변환해주고 padding
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [12]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)

(54879, 500) (19617, 500)


## Stratified K-Fold Cross Validation
*Stratified N-Fold CV: N-Fold CV에서 각각의 폴드에서 종속변수의 분포가 동일하도록 폴드를 나누는 방식.
현재 사용하는 데이터처럼 분류학습에서 종속변수의 범주의 분포가 균일하지 않을 때 사용된다.

In [13]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [14]:
#가벼운 NLP모델 생성
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(.5),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),    
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(.5),
        Dense(n_class, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.0003))
    return model

In [15]:
# fit model & predict values
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                            patience=3, min_lr=1e-6, mode='min', verbose=1)

    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            callbacks=[es, rlr])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Epoch 1/10
1372/1372 [==============================] - 106s 77ms/step - loss: 1.4698 - val_loss: 1.2688
Epoch 2/10
1372/1372 [==============================] - 111s 81ms/step - loss: 1.1108 - val_loss: 0.9706
Epoch 3/10
1372/1372 [==============================] - 116s 84ms/step - loss: 0.8542 - val_loss: 0.8316
Epoch 4/10
1372/1372 [==============================] - 117s 85ms/step - loss: 0.6798 - val_loss: 0.7926
Epoch 5/10
1372/1372 [==============================] - 115s 84ms/step - loss: 0.5724 - val_loss: 0.7738
Epoch 6/10
1372/1372 [==============================] - 105s 76ms/step - loss: 0.4960 - val_loss: 0.7751
Epoch 7/10
1372/1372 [==============================] - 113s 82ms/step - loss: 0.4384 - val_loss: 0.7848
Epoch 8/10
1372/1372 [==============================] - ETA: 0s - loss: 0.3994- ETA: 0s - loss: 0.3Restoring model weights from the end of the best epoch.

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
1372

In [16]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  71.5264%
Log Loss (CV):   0.7777


In [17]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

## 시각화

In [18]:
# model summary
print(clf.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 64)           1280000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 165, 128)          57472     
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 53, 128)           114816    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)              

In [19]:
#plot_model(clf)

## 제출 파일 생성

In [20]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [21]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0424,0.5989,1.0766e-01,2.2232e-01,0.0287
1,0.1915,0.5356,4.9047e-02,3.7313e-02,0.1865
2,0.9987,0.0008,1.2554e-05,7.0363e-06,0.0005
3,0.0009,0.0003,7.8165e-01,3.4634e-04,0.2169
4,0.7184,0.0222,6.9613e-02,4.8803e-02,0.1410


In [22]:
sub.to_csv(sub_file)